<a href="https://colab.research.google.com/github/gurovic/MLCourse/blob/main/103_knn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Подготовка среды

In [ ]:
!pip install scikit-learn matplotlib numpy pandas
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.datasets import load_iris, make_moons, make_regression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay, mean_squared_error
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import time

## Теоретическое введение

kNN - один из простейших алгоритмов машинного обучения:
- **Принцип работы**: классификация объектов по большинству голосов k ближайших соседей
- **Гиперпараметры**:
  - `n_neighbors` (k) - количество соседей
  - `weights` - вес голосов (равные или по расстоянию)
  - `metric` - метрика расстояния (евклидово, манхэттенское и др.)
- **Преимущества**: простота, интерпретируемость, нет этапа обучения
- **Недостатки**: медленный на больших данных, чувствителен к масштабу признаков

**Алгоритм kNN**:
1. Для нового объекта найти k ближайших соседей в обучающей выборке
2. Определить класс по большинству голосов среди соседей
3. Предсказать класс

### Создание демо-данных

In [ ]:
# Загрузка классического набора Iris
iris = load_iris()
X_iris = iris.data[:, :2]  # Берем только 2 признака для визуализации
y_iris = iris.target

# Генерация сложных данных "Луны"
X_moons, y_moons = make_moons(n_samples=500, noise=0.3, random_state=42)

# Создание DataFrame для Iris
iris_df = pd.DataFrame(X_iris, columns=['sepal_length', 'sepal_width'])
iris_df['species'] = y_iris
iris_df['species'] = iris_df['species'].map({0: 'setosa', 1: 'versicolor', 2: 'virginica'})

print("Первые 5 строк данных Iris:")
display(iris_df.head())

# Визуализация данных Iris
plt.figure(figsize=(8, 6))
for species, color in zip(['setosa', 'versicolor', 'virginica'], ['red', 'green', 'blue']):
    subset = iris_df[iris_df['species'] == species]
    plt.scatter(subset['sepal_length'], subset['sepal_width'], c=color, label=species)
plt.title('Данные Iris')
plt.xlabel('Длина чашелистика (см)')
plt.ylabel('Ширина чашелистика (см)')
plt.legend()
plt.grid(True)
plt.show()

## 🟢 Базовый уровень

### 1.1 Простая реализация kNN

In [ ]:
# Разделение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(
    X_iris, y_iris, test_size=0.25, random_state=42
)

# Создание и обучение модели
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)

# Предсказание и оценка
y_pred = knn.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Точность модели: {accuracy:.2f}")

# Матрица ошибок
cm = confusion_matrix(y_test, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=iris.target_names)
disp.plot(cmap='Blues')
plt.title('Матрица ошибок (Iris)')
plt.show()

### 1.2 Визуализация решающих областей

In [ ]:
def plot_decision_boundary(X, y, model, title):
    h = .02  # Шаг сетки
    cmap_light = ListedColormap(['#FFAAAA', '#AAFFAA', '#AAAAFF'])
    cmap_bold = ListedColormap(['#FF0000', '#00FF00', '#0000FF'])

    # Границы графика
    x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1

    # Создание сетки
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                         np.arange(y_min, y_max, h))

    # Предсказание для каждой точки сетки
    Z = model.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)

    # Визуализация
    plt.figure(figsize=(10, 6))
    plt.pcolormesh(xx, yy, Z, cmap=cmap_light, shading='auto')
    plt.scatter(X[:, 0], X[:, 1], c=y, cmap=cmap_bold, edgecolor='k', s=20)
    plt.xlim(xx.min(), xx.max())
    plt.ylim(yy.min(), yy.max())
    plt.title(title)
    plt.xlabel('Длина чашелистика')
    plt.ylabel('Ширина чашелистика')
    plt.show()

plot_decision_boundary(X_iris, y_iris, knn, "kNN (k=5) для данных Iris")

## 🟡 Продвинутый уровень

### 2.1 Влияние количества соседей (k)

In [ ]:
plt.figure(figsize=(12, 8))
k_values = [1, 5, 15, 30]
accuracies = []

for i, k in enumerate(k_values):
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_train, y_train)
    y_pred = knn.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    accuracies.append(acc)

    plt.subplot(2, 2, i+1)

    # Границы графика
    x_min, x_max = X_iris[:, 0].min() - 1, X_iris[:, 0].max() + 1
    y_min, y_max = X_iris[:, 1].min() - 1, X_iris[:, 1].max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.02),
                         np.arange(y_min, y_max, 0.02))

    # Предсказание
    Z = knn.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)

    # Визуализация
    plt.pcolormesh(xx, yy, Z, cmap=plt.cm.Paired, shading='auto', alpha=0.3)
    plt.scatter(X_iris[:, 0], X_iris[:, 1], c=y_iris, edgecolor='k', s=20)
    plt.xlim(xx.min(), xx.max())
    plt.ylim(yy.min(), yy.max())
    plt.title(f"k = {k} (Точность: {acc:.2f})")
    plt.xlabel('Длина чашелистика')
    plt.ylabel('Ширина чашелистика')

plt.tight_layout()
plt.show()

# График зависимости точности от k
plt.figure(figsize=(10, 5))
plt.plot(k_values, accuracies, 'bo-', linewidth=2)
plt.xlabel('Количество соседей (k)')
plt.ylabel('Точность')
plt.title('Зависимость точности от количества соседей')
plt.grid(True)
plt.show()

### 2.2 Важность масштабирования признаков

In [ ]:
# Без масштабирования
knn_raw = KNeighborsClassifier(n_neighbors=5)
knn_raw.fit(X_train, y_train)
y_pred_raw = knn_raw.predict(X_test)
accuracy_raw = accuracy_score(y_test, y_pred_raw)

# С масштабированием
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

knn_scaled = KNeighborsClassifier(n_neighbors=3)
knn_scaled.fit(X_train_scaled, y_train)
y_pred_scaled = knn_scaled.predict(X_test_scaled)
accuracy_scaled = accuracy_score(y_test, y_pred_scaled)

print(f"Точность без масштабирования: {accuracy_raw:.2f}")
print(f"Точность с масштабированием: {accuracy_scaled:.2f}")

# Визуализация масштабированных данных
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.scatter(X_train[:, 0], X_train[:, 1], c=y_train)
plt.title("Исходные данные")
plt.xlabel("Признак 1")
plt.ylabel("Признак 2")

plt.subplot(1, 2, 2)
plt.scatter(X_train_scaled[:, 0], X_train_scaled[:, 1], c=y_train)
plt.title("Масштабированные данные")
plt.xlabel("Признак 1 (стандартизированный)")
plt.ylabel("Признак 2 (стандартизированный)")
plt.tight_layout()
plt.show()

### 2.3 Подбор гиперпараметров с GridSearchCV

In [ ]:
# Создание параметров для перебора
param_grid = {
    'n_neighbors': list(range(1, 31)),
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan']
}

# Поиск по сетке
grid = GridSearchCV(
    KNeighborsClassifier(),
    param_grid,
    cv=5,
    scoring='accuracy',
    n_jobs=-1
)
grid.fit(X_train_scaled, y_train)

# Лучшие параметры
print(f"Лучшие параметры: {grid.best_params_}")
print(f"Лучшая точность: {grid.best_score_:.2f}")

# Визуализация результатов
results = pd.DataFrame(grid.cv_results_)
plt.figure(figsize=(12, 6))
for metric in ['euclidean', 'manhattan']:
    subset = results[results['param_metric'] == metric]
    plt.plot(subset['param_n_neighbors'], subset['mean_test_score'],
             label=f"{metric} distance")

plt.xlabel('Количество соседей (k)')
plt.ylabel('Точность')
plt.title('Влияние параметров k и метрики на точность')
plt.legend()
plt.grid(True)
plt.show()

## 🔴 Экспертный уровень

### 3.1 Работа с нелинейными данными

In [ ]:
# Разделение данных "Луны"
X_train_m, X_test_m, y_train_m, y_test_m = train_test_split(
    X_moons, y_moons, test_size=0.3, random_state=42
)

# Обучение модели
knn_moons = KNeighborsClassifier(n_neighbors=15, weights='distance')
knn_moons.fit(X_train_m, y_train_m)

# Визуализация
plot_decision_boundary(X_moons, y_moons, knn_moons, "kNN для данных 'Луны'")

# Точность
accuracy_moons = knn_moons.score(X_test_m, y_test_m)
print(f"Точность на данных 'Луны': {accuracy_moons:.2f}")

### 3.2 Оптимизация вычислений с KD-Tree

In [ ]:
# Создание больших данных
X_large, y_large = make_moons(n_samples=1000000, noise=0.2, random_state=42)

# Сравнение алгоритмов
algorithms = ['auto', 'ball_tree', 'kd_tree', 'brute']
times = []

for algo in algorithms:
    start_time = time.time()
    knn_large = KNeighborsClassifier(n_neighbors=5, algorithm=algo)
    knn_large.fit(X_large, y_large)
    y_pred_large = knn_large.predict(X_large[:1000])  # Предсказание для подмножества
    elapsed = time.time() - start_time
    times.append(elapsed)
    print(f"Алгоритм: {algo:<10} Время: {elapsed:.4f} сек")

# Визуализация
plt.figure(figsize=(10, 5))
plt.bar(algorithms, times, color=['blue', 'green', 'red', 'purple'])
plt.title('Сравнение скорости алгоритмов kNN')
plt.ylabel('Время выполнения (сек)')
plt.xlabel('Тип алгоритма')
plt.show()

### 3.3 Применение в задачах регрессии

In [ ]:
# Генерация данных
np.random.seed(42)
X_reg = np.sort(5 * np.random.rand(200, 1), axis=0)
y_reg = np.sin(X_reg).ravel() + np.random.normal(0, 0.1, X_reg.shape[0])

# Обучение моделей
k_values = [1, 5, 15]
plt.figure(figsize=(15, 5))
mse_scores = []

for i, k in enumerate(k_values):
    # Обучение
    knn_reg = KNeighborsRegressor(n_neighbors=k)
    knn_reg.fit(X_reg, y_reg)

    # Предсказание
    X_test_reg = np.linspace(0, 5, 500)[:, np.newaxis]
    y_pred_reg = knn_reg.predict(X_test_reg)

    # Оценка
    mse = mean_squared_error(y_reg, knn_reg.predict(X_reg))
    mse_scores.append(mse)

    # Визуализация
    plt.subplot(1, 3, i+1)
    plt.scatter(X_reg, y_reg, color='blue', alpha=0.5, label='Данные')
    plt.plot(X_test_reg, y_pred_reg, color='red', linewidth=2, label='Предсказание')
    plt.title(f'kNN Регрессия (k={k}, MSE={mse:.4f})')
    plt.legend()
    plt.grid(True)

plt.tight_layout()
plt.show()

# График зависимости MSE от k
plt.figure(figsize=(8, 4))
plt.plot(k_values, mse_scores, 'ro-', linewidth=2)
plt.xlabel('Количество соседей (k)')
plt.ylabel('MSE')
plt.title('Зависимость ошибки от количества соседей в регрессии')
plt.grid(True)
plt.show()

## 📊 Чеклист по уровням

| Уровень  | Навыки |
|----------|--------|
| 🟢       | Базовое применение kNN, визуализация решающих областей |
| 🟡       | Подбор гиперпараметров, важность масштабирования, GridSearchCV |
| 🔴       | Работа с нелинейными данными, оптимизация алгоритмов, kNN регрессия |

## ⚠️ Критические ошибки

1. **Неправильный выбор k**: слишком маленькое k → переобучение, слишком большое → недообучение  
2. **Игнорирование масштабирования**: без него признаки с большим разбросом будут доминировать  
3. **Использование на больших данных**: kNN неэффективен для больших датасетов (>50K объектов)  
4. **Работа с категориальными признаками**: kNN требует преобразования в числовой формат

## 💡 Главные принципы

1. **Всегда масштабируйте данные** перед использованием kNN
2. **Оптимизируйте k** с помощью кросс-валидации (начните с k = √n)
3. **Экспериментируйте с метриками расстояния**:
   - Евклидово: √Σ(x_i - y_i)² (стандартное)
   - Манхэттенское: Σ|x_i - y_i| (устойчивее к выбросам)
4. **Используйте взвешенные голоса** (weights='distance') для уменьшения влияния далеких соседей
5. **Рассмотрите уменьшение размерности** для наборов с большим количеством признаков

## 📌 Итог

kNN - мощный и интуитивно понятный алгоритм:
- **Простота**: легко понять и реализовать
- **Универсальность**: работает для классификации и регрессии
- **Интерпретируемость**: предсказания основаны на реальных объектах

> 💡 Правило: "kNN - отличный выбор для небольших датасетов и задач с явной геометрической интерпретацией, но требует тщательной подготовки данных и настройки гиперпараметров."

**Этапы работы с kNN**:
1. Масштабирование признаков
2. Выбор k и метрики расстояния
3. Обучение модели
4. Оценка качества
5. Прогнозирование

**Практическое задание**:
1. Для данных Iris:
   - Найдите оптимальное k с кросс-валидацией
   - Сравните точность с масштабированием и без
   - Визуализируйте решающие границы для лучшей модели

2. Для данных "Луны":
   - Подберите оптимальные параметры с GridSearchCV
   - Сравните разные метрики расстояния
   - Постройте кривую обучения для разного размера выборки

3. Реализуйте kNN регрессию для предсказания цен на жилье:
   - Используйте датасет Boston Housing
   - Сравните разные значения k
   - Оцените качество с помощью MSE и R²